In [1]:
# some time steps in the forcing may be problematic
# this step replaces those files

import xarray as xr
import numpy as np
import glob, os
import cftime
import pandas as pd

In [2]:
month = '1979-01'

In [3]:
infile = '/glade/campaign/cgd/tss/common/lm_forcing/hybrid/era5land_eme/Precip/clmforc.E5LEME.c2023.010x010.Precip.1979-01.nc'

In [4]:
tarfiles = []
for i in range(30):
    filei = f'/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_{i}_SubsetForcing/Precip/subset_clmforc.E5LEME.c2023.010x010.Precip.1975-1979.nc'
    tarfiles.append(filei)
    
for i in range(31, 37):
    filei = f'/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_{i}_SubsetForcing/Precip/subset_clmforc.E5LEME.c2023.010x010.Precip.1975-1979.nc'
    tarfiles.append(filei)

tarfiles.append(f'/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_30_SubsetForcing/Precip/subset_clmforc.E5LEME.c2023.010x010.Precip.1976-1980.nc')

for f in tarfiles:
    if not os.path.isfile(f):
        print(f)

In [5]:
# backup
tarfiles2 = []
for f in tarfiles:
    f2 = f+'-original'
    tarfiles2.append(f2)
    if not os.path.isfile(f2):
        os.system(f'cp {f} {f2}')
        

In [10]:
# Load the replacement data for the specific month
replacement_data = xr.open_dataset(infile).sel(time=month)
lat_all = replacement_data['LATIXY'].values[:,0]
lon_all = replacement_data['LONGXY'].values[0,:]

# Process each target file
for tarfile,outfile in zip(tarfiles2,tarfiles):
    print('Processing:', tarfile)
    if os.path.isfile(tarfile):
        with xr.open_dataset(tarfile) as ds:

            # Determine the subset of lat/lon from infile that matches target file
            lat_subset = ds['LATIXY'].values[:,0]
            lon_subset = ds['LONGXY'].values[0,:]
            indlat = [np.where(lat_all==i)[0][0] for i in lat_subset]
            indlon = [np.where(lon_all==i)[0][0] for i in lon_subset]

            # Select the matching subset from the replacement data
            replacement_subset = replacement_data.isel(lat=indlat, lon=indlon)
            
            # Replace the data for the specified month
            target_month = pd.to_datetime(month)
            ym = [f'{y}-{m:02}' for y, m in zip(ds['time'].dt.year.values, ds['time'].dt.month.values)]
            indices = np.array([i for i, date in enumerate(ym) if date == "1979-01"])
            
            replacement_precip = replacement_subset['PRECTmms']
            values = ds['PRECTmms'].values
            values[indices, :, :] = replacement_precip
            ds['PRECTmms'].values = values
            
            dd2 = ds['PRECTmms'].loc[dict(time=month)].values.copy()
            if np.sum(np.isnan(dd2))>0:
                sys.exit("Still NAN!")
            
            # Save the modified file
            print('Saving to:',outfile)
            ds.to_netcdf(outfile, format='NETCDF3_CLASSIC')
    else:
        print(f'File not found: {tarfile}')


Processing: /glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_0_SubsetForcing/Precip/subset_clmforc.E5LEME.c2023.010x010.Precip.1975-1979.nc-original
Saving to: /glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_0_SubsetForcing/Precip/subset_clmforc.E5LEME.c2023.010x010.Precip.1975-1979.nc
Processing: /glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_1_SubsetForcing/Precip/subset_clmforc.E5LEME.c2023.010x010.Precip.1975-1979.nc-original
Saving to: /glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_1_SubsetForcing/Precip/subset_clmforc.E5LEME.c2023.010x010.Precip.1975-1979.nc
Processing: /glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_2_SubsetForcing/Precip/subset_clmforc.E5LEME.c2023.010x010.Precip.1975-1979.nc-original
Saving to: /glade/work/guoqiang/CTSM_cases/CAMELS_Calib/Calib_all_HH_MOASMO/level1_2_SubsetForcing/Precip/subset_clmforc.E5LEME.c2023.010x010.Precip.1975-1979.n